```
* 5112개의 비낙상 데이터와 5096개의 낙상 데이터를 데이터셋으로 옮김 (낙상 데이터가 과도하게 많아 기존 15288개 중 1/3인 5096개만 사용하기로 함)
* 1 피실험자당 8개의 카메라 각도로 촬영됨
* 1개의 카메라 각도당 이미지는 10장, 비디오는 10초 촬영됨 (10초/비디오 * 60 프레임/초 = 600 프레임)
* yolov8s 학습에는 데이터셋의 모든 이미지를 사용함 (10208 * 10 = 102080 사진)
* GRU 학습에는 데이터셋의 1/5에 해당하는 비디오만 사용함 (2024 동영상)
* 동영상은 1개당 600 프레임이며, 6프레임 별로 yolo bbox ratio 1개 및 mediapipe 관절 좌표값 22개를 추출함 (100 프레임/동영상 * 23개/프레임 = 2300개/동영상)

* 기존 label에 기록된 낙상 중인 시점을 이용하여 100 프레임/동영상에 대해 각 프레임당 피실험자 상태(정상/낙상 중/낙상 후)를 classification함
* 2024개의 동영상(100 프레임/동영상)에 대해 yolo bbox, mediapipe 관절 좌표값을 탐지하여 JSON으로 추출함 (팀원 GPU 이용)
* 우리는 이미 추출된 JSON에 맞춰 동영상 데이터셋을 생성하기로 함
```

In [9]:
import os
import glob
import random
import shutil
from tqdm import tqdm
from collections import defaultdict

In [10]:
# 랜덤 시드 설정
RANDOM_SEED = 2024
random.seed(RANDOM_SEED)

In [11]:
# 이미 추출된 JSON 경로 설정
json_combined_dir = 'D:\\Falldown\\Json_combined'

In [12]:
# 이미 추출된 JSON을 다시 split할 경로
train_combined_json_dir = 'D:\\Falldown\\Dataset\\Video_Dataset\\Json_combined\\Train'
val_combined_json_dir = 'D:\\Falldown\\Dataset\\Video_Dataset\\Json_combined\\Val'
test_combined_json_dir = 'D:\\Falldown\\Dataset\\Video_Dataset\\Json_combined\\Test'

# 이미 추출된 combined_JSON에 맞춘 동영상 데이터셋 경로
train_target_video_dir = 'D:\\Falldown\\Dataset\\Video_Dataset\\Video\\Train'
val_target_video_dir = 'D:\\Falldown\\Dataset\\Video_Dataset\\Video\\Val'
test_target_video_dir = 'D:\\Falldown\\Dataset\\Video_Dataset\\Video\\Test'

# 이미 추출된 combined_JSON에 맞춘 initial JSON 데이터셋 경로
train_initial_json_dir = 'D:\\Falldown\\Dataset\\Video_Dataset\\Json_initial\Train'
val_initial_json_dir = 'D:\\Falldown\\Dataset\\Video_Dataset\\Json_initial\Val'
test_initial_json_dir = 'D:\\Falldown\\Dataset\\Video_Dataset\\Json_initial\Test'

In [13]:
# Source 비디오 파일 경로 패턴
video_pattern = r'D:\\Falldown\\Dataset\\Original_Dataset\\*\\SourceData\\Video\\*\\*\\*\\*.mp4'

# Source JSON 파일 경로 패턴
json_pattern = r'D:\\Falldown\\Origin\\*\\LabelingData\\Video\\*\\*\\*\\*.json'

In [21]:
# 필요한 디렉토리 생성
for target_dir in [
    train_combined_json_dir, val_combined_json_dir, test_combined_json_dir,
    train_target_video_dir, val_target_video_dir, test_target_video_dir,
    train_initial_json_dir, val_initial_json_dir, test_initial_json_dir
]:
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

In [22]:
# 그룹화
json_files = glob.glob(os.path.join(json_combined_dir, '**', '*.json'), recursive=True)
grouped_files = defaultdict(list)

for file_path in json_files:
    # 그룹키는 파일 이름에서 환자 단위로 추출 (ex. "00060_H_A_SY")
    group_key = os.path.basename(file_path).rsplit('_C', 1)[0]
    grouped_files[group_key].append(file_path)

In [23]:
# 그룹 키 리스트 추출 및 셔플
group_keys = list(grouped_files.keys())
random.shuffle(group_keys)

# 그룹 키를 Train, Validation, Test로 분할
def split_groups(group_keys, split_ratios=(0.8, 0.1, 0.1)):
    total_groups = len(group_keys)
    train_end = int(total_groups * split_ratios[0])
    val_end = train_end + int(total_groups * split_ratios[1])
    
    train_keys = group_keys[:train_end]
    val_keys = group_keys[train_end:val_end]
    test_keys = group_keys[val_end:]
    
    return train_keys, val_keys, test_keys

train_keys, val_keys, test_keys = split_groups(group_keys)

In [25]:
print(f"총 JSON 파일 수: {len(json_files)}")
print(f"총 그룹 수: {len(grouped_files)}")

총 JSON 파일 수: 2024
총 그룹 수: 253


In [24]:
# Combined JSON 복사 함수
def copy_json_files(group_keys, json_target_dir):
    for group_key in tqdm(group_keys, desc=f"{json_target_dir.split(os.sep)[-2]} JSON 복사 중", unit="group"):
        for json_file in grouped_files[group_key]:
            shutil.copy(json_file, os.path.join(json_target_dir, os.path.basename(json_file)))

# 동영상 복사 함수
def copy_video_files(group_keys, video_pattern, video_target_dir):
    video_files = glob.glob(video_pattern, recursive=True)
    for group_key in tqdm(group_keys, desc=f"{video_target_dir.split(os.sep)[-2]} 동영상 복사 중", unit="group"):
        for video_file in video_files:
            if group_key in os.path.basename(video_file):
                shutil.copy(video_file, os.path.join(video_target_dir, os.path.basename(video_file)))

# Initial JSON 복사 함수
def copy_initial_json_files(group_keys, json_pattern, initial_json_target_dir):
    json_source_files = glob.glob(json_pattern, recursive=True)
    for group_key in tqdm(group_keys, desc=f"{initial_json_target_dir.split(os.sep)[-2]} Initial JSON 복사 중", unit="group"):
        for json_file in json_source_files:
            if group_key in os.path.basename(json_file):
                shutil.copy(json_file, os.path.join(initial_json_target_dir, os.path.basename(json_file)))

# 데이터셋 복사 시작
print("Combined JSON 데이터셋 복사 시작")
copy_json_files(train_keys, train_combined_json_dir)
copy_json_files(val_keys, val_combined_json_dir)
copy_json_files(test_keys, test_combined_json_dir)

print("Video 데이터셋 복사 시작")
copy_video_files(train_keys, video_pattern, train_target_video_dir)
copy_video_files(val_keys, video_pattern, val_target_video_dir)
copy_video_files(test_keys, video_pattern, test_target_video_dir)

print("Initial JSON 데이터셋 복사 시작")
copy_initial_json_files(train_keys, json_pattern, train_initial_json_dir)
copy_initial_json_files(val_keys, json_pattern, val_initial_json_dir)
copy_initial_json_files(test_keys, json_pattern, test_initial_json_dir)

# 결과 출력
print(f"Train: {len(train_keys)} groups")
print(f"Val: {len(val_keys)} groups")
print(f"Test: {len(test_keys)} groups")

Combined JSON 데이터셋 복사 시작


Json_combined JSON 복사 중: 100%|██████████| 26/26 [00:00<00:00, 155.48group/s]


Video 데이터셋 복사 시작


Video 동영상 복사 중: 100%|██████████| 26/26 [00:12<00:00,  2.13group/s]


Initial JSON 데이터셋 복사 시작


Json_initial Initial JSON 복사 중: 100%|██████████| 26/26 [00:00<?, ?group/s]

Train: 202 groups
Val: 25 groups
Test: 26 groups
